In [1]:
# run_id_predictor

# training set: June 2018 bus locations =~ 100,000 position observations for any given line

# features: given v (bus id), route, timestamp
# target: predict which schedule run the bus is on ('run')

# n.b. the run_id is omitted from the getStopPredictions.jsp API response for inbound buses to the stop

# n.b. the run should be but doesn't map up against the GTFS trip_id (is there a lookup table?)


In [2]:
# # config 1
# source = 'nj'
# route = 119
# stop_no = 30189 # webster and congess, jersey city

In [3]:
# config 2
source = 'nj'
route = 87
stop_no = 21062 # palisade and south, jersey city

### get training data

In [4]:
# relative import
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from src.buses.reportcard_helpers import *

In [5]:
# get the training dataset - takes about a minute for Rt. 119
# how long for entire database?

from mysql.connector import connection
db_user ='buswatcher'
db_password = 'njtransit'
db_host = 'localhost'
db_name = 'bus_position_log'
conn = connection.MySQLConnection(user=db_user, password=db_password, host=db_host, database=db_name)

arrival_query = ('SELECT lat,lon,bid,rt,timestamp,run FROM run_predictor_training_set WHERE (rt="%s");' % route)
df = pd.read_sql_query(arrival_query, conn)


In [6]:
# cleanup crew ------------------------------------------------------------------------

# recode 'manager' runs
df.replace(
        to_replace='MAN',
        value=unicode('666'), # vs value=np.NaN,
        inplace=True,
        limit=None,
        regex=False, 
        method='pad')

# fix the timestamp
df['timestamp'] = df['timestamp'].str.split('.').str.get(0)
df = df.set_index(pd.DatetimeIndex(df['timestamp']), drop=False)

# extract the hh:mm:ss part of timestamp
df['timestamp_ml'] = df.index.time
df['timestamp_ml'] = df['timestamp_ml'].apply(lambda x: float(str(x).replace(":","")))

# get rid of the timestamp and route
df = df.reset_index(drop=True)
df = df.drop(columns=['timestamp','rt'])

# change bid to v for consistency with stopwatcher
df.rename(columns = {'bid':'v'}, inplace = True)

# straggler strings in run
df['run'] = df['run'].str.replace(r'\D+', '')

# negatives in longitude
df['lon'] = abs(df['lon'])

# make run a number for easier modeling
df['run'] = df['run'].apply(pd.to_numeric)

# reorder columns
df = df[['run', 'timestamp_ml','lat','lon','v']]


In [7]:
# inspect
df.head()

,run,timestamp_ml,lat,lon,v
0,244,191839.0,40.739040,74.042519,6960
1,10,191839.0,40.748774,74.040556,6852
2,257,191839.0,40.711076,74.078812,6943
3,239,191839.0,40.730632,74.063024,6963
4,236,191839.0,40.726005,74.067727,6938


### encode features

In [8]:
# pandas encoder
df = pd.get_dummies(df, columns=['v'])

In [9]:
# inspect
df.head()

,run,timestamp_ml,lat,lon,v_10943,v_10944,v_10945,v_10946,v_10947,v_10956,...,v_6973,v_6974,v_892,v_893,v_894,v_895,v_903,v_904,v_905,v_906
0,244,191839.0,40.739040,74.042519,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,191839.0,40.748774,74.040556,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,257,191839.0,40.711076,74.078812,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,239,191839.0,40.730632,74.063024,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,236,191839.0,40.726005,74.067727,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# X,y and train-test splits

# just bus #
# lr 1 / 1
# rf .72 / .72
# nn 1 / 1
# X = df.filter(regex='(^v_.*$)', axis=1) # regex 

# bus and timestamp
# lr .07
# rf .74
# nn 1.0
# X = df.filter(regex='(^v_.*$|timestamp_ml)', axis=1) # regex 

# just lat lon timestamp
# lr .01
# rf .26
# nn DNF
# X = df.filter(regex='(lat|lon|timestamp_ml)', axis=1) # regex 

# everything
# lr .1
# rf .71
# nn 1.0
X = df.filter(regex='(lat|lon|^v_.*$|rt|timestamp_ml)', axis=1) # regex 

y= df.filter(regex='(run)', axis=1) # regex for v,rt,run


In [11]:
X.head()

,timestamp_ml,lat,lon,v_10943,v_10944,v_10945,v_10946,v_10947,v_10956,v_10957,...,v_6973,v_6974,v_892,v_893,v_894,v_895,v_903,v_904,v_905,v_906
0,191839.0,40.739040,74.042519,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,191839.0,40.748774,74.040556,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,191839.0,40.711076,74.078812,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,191839.0,40.730632,74.063024,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,191839.0,40.726005,74.067727,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
y.head()

,run
0,244
1,10
2,257
3,239
4,236


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split (X,y, test_size=0.2, random_state=42)

### models

In [14]:
# logistic regression

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train.values.ravel())

from sklearn.metrics import accuracy_score
print (">>One-Hot Encoding (via pandas): Logistic Regression<<")
print "Train Accuracy :: ", accuracy_score(y_train, lr.predict(X_train))
print "Test Accuracy  :: ", accuracy_score(y_test, lr.predict(X_test))
print ("Accuracy on training set: {:.3f}".format(lr.score(X_train,y_train)))  # AT add
print ("Accuracy on test set: {:.3f}".format(lr.score(X_test,y_test))) # AT add


>>One-Hot Encoding (via pandas): Logistic Regression<<
Train Accuracy ::  0.10039722525253367
Test Accuracy  ::  0.09926384863928583
Accuracy on training set: 0.100
Accuracy on test set: 0.099


In [15]:
### random forest 

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=25,max_depth=10)
rf.fit(X_train,y_train.values.ravel())

print (">>One-Hot Encoding (via pandas): Random Forest<<")
print ("Accuracy on training set: {:.3f}".format(rf.score(X_train,y_train)))  # AT add
print ("Accuracy on test set: {:.3f}".format(rf.score(X_test,y_test))) # AT add



>>One-Hot Encoding (via pandas): Random Forest<<
Accuracy on training set: 0.710
Accuracy on test set: 0.711


In [16]:
### neural net

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
nn.fit(X_train,y_train.values.ravel())

print (">>One-Hot Encoding (via pandas): Neural Net<<")
print ("Accuracy on training set: {:.3f}".format(nn.score(X_train,y_train)))  # AT add
print ("Accuracy on test set: {:.3f}".format(nn.score(X_test,y_test))) # AT add

# from sklearn.metrics import classification_report
# predictions = nn.predict(X_test)
# print(classification_report(y_test,predictions))

>>One-Hot Encoding (via pandas): Neural Net<<
Accuracy on training set: 1.000
Accuracy on test set: 1.000


### make some predictions on field observations

In [17]:
# get the arrival predictions for NOW for this STOP
from src.buses.Buses import *
import datetime
now = datetime.datetime.now()
arrivals = parse_stopprediction_xml(get_xml_data(source, 'stop_predictions', stop=stop_no, route=route))
arrivals

[StopPrediction[cars= consist= fd=87 HOBOKEN-PATH VIA JOURNAL SQ m=1 name=StopPrediction pt=27 rd=87 rn=87 scheduled=false stop_id=21062 stop_name=PALISADE AVE + SOUTH ST v=5720]]

In [40]:
# make arrivals look like X_train

df_arrivals = pd.DataFrame(columns=['timestamp_ml','v'])

data = []
for bus in arrivals:
    bus.timestamp_ml = float(str(now.hour)+str(now.minute)+str(now.second))
    keys = ['timestamp_ml','v']
    values = [bus.timestamp_ml,bus.v]
    data.append(dict(zip(keys,values)))
df_arrivals = pd.DataFrame(data)
df_arrivals

,timestamp_ml,v
0,204344.0,5720


In [43]:
# encode arrivals_temp with the same encoder used on training data

# https://stackoverflow.com/questions/28465633/easy-way-to-apply-transformation-from-pandas-get-dummies-to-new-data
# df = pd.DataFrame({'cat':['a','b','c','d'],'val':[1,2,5,10]})
df1 = pd.get_dummies(pd.DataFrame(df_arrivals))
dummies_frame = pd.get_dummies(df)
df1.reindex(columns = dummies_frame.columns, fill_value=0)
df1


,timestamp_ml,v_5720
0,204344.0,1


In [45]:
# predictions

import numpy as np

for bus in df1:
    bus = np.asarray(bus)
    bus = bus.reshape(1, -1)
    print 'Logistic Regression predictions run: %s' % (lr.predict(bus))
    print 'Random Forest predictions run: %s' % (rf.predict(bus))
    print 'Neural Net predictions run: %s' % (nn.predict(bus))

ValueError: X has 1 features per sample; expecting 156

In [ ]:
#BREAK-------------------------------------------------------------------------------------------------------
import sys
sys.exit()

In [ ]:
# desired output

# I see 2 buses approaching.
# Bus #4532 is 5 minutes away and I think its on run #505. That makes it 10 minutes late.
# Bus #7330 is 19 minutes away and I think its on run #525. That makes it 3 minutes late.


In [ ]:
# if the model is working
# pickle it

import pickle
s = pickle.dumps(r)
s2 = pickle.loads(r)
r2.predict(X[0:1])
array([0])
y[0]
0